<a href="https://colab.research.google.com/github/LuizGusQueiroz/Projetos_Estagio/blob/main/FolhasPagamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração do ambiente

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import html
import os
import re
from tqdm import tqdm

# Pré Processamento

In [68]:
def clean_html(arq_html):
  # Salva apenas as linhas que tem 'class'
  # e aplica a formatação para adicionar acentos
  arq_html = [html.unescape(row) for row in arq_html if 'class' in row]
  # Remove as divs
  for i, row in enumerate(arq_html):
    pattern = r'<[^>]*>'
    new_row = re.sub(pattern, '', row)
    arq_html[i] = new_row

  return arq_html

In [104]:
def get_registros_from_html(path, nome_arq='FolhaPagamento'):

  # Abre o arquivo html
  with open(path, 'r') as file:
    arq_html = file.read()
  arq_html = arq_html.split('\n')

  # Aplica uma função limpa o arquivo
  arq_html = clean_html(arq_html)

  # Declaração de variáveis
  columns = ['Competência','Empregador','Código_Empregado','Nome_Empregado','Cargo','Lotação','Código_Evento','Desc_Evento','Provento','Desconto']
  registros = pd.DataFrame(columns=columns)
  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  lotacao = str()
  cargo = str()

  # Total de empregados que serão coletados:
  total_func = 0
  for row in arq_html:
    if row == 'Data e Assinatura':
      total_func += 1

  # A variável start define o começo dos dados de cada empregado
  start = 0

  for _ in range(total_func):

    # A coleta destas informações não é feita por uso de um 'for'
    # pois a depender da empresa, alguma informação pode mudar
    # de posição

    # Encontra a competência
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Competência':
        # A competência está uma linha após esta palavra
        competencia = arq_html[i+1]
        break
    # Encontra o empregador
    for i, row in enumerate(arq_html[start:], start=start):
      if row ==  'Empregador':
        # O empregador está uma linha após a palavra
        empregador = arq_html[i+1]
        break
    # Encontra a lotação
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Lotação':
        # A lotação está uma linha após a palavra
        lotacao = arq_html[i+1]
        break
    # Encontra o cargo
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Cargo':
        # O cargo está uma linha após a palavra
        cargo = arq_html[i+1]
        break
    # Encontra o código e nome do empregado
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Empregado':
        # O código e nome estão uma linha após a palavra
        cod_e_nome = arq_html[i+1].split()
        codigo_emp = cod_e_nome[0]
        nome_emp = ' '.join(cod_e_nome[1:])
        break
    # Guarda os eventos do funcionário
    index = 0
    eventos = list()
    registro = list()

    for i, row in enumerate(arq_html[start:], start=start):
      # Os proventos começam após a linha com 'Cod.'
      if row == 'Cod.':
        index = i+1
        break


    while arq_html[index] != 'Total de Proventos':
      registro.append(arq_html[index])
      index += 1

      if len(registro) == 5:
        # Verifica se o código é não nulo
        if arq_html[index-1]:
          eventos.append(registro)
          registro = list()
        # Caso seja nulo, os eventos acabaram
        else:
          break

    # Converte a lista de eventos para DataFrame
    eventos = pd.DataFrame(eventos)
    # Pode acontecer de aparecer um funcionário sem registros
    # caso esteja afastado, então será ignorado
    if not eventos.empty:
      # Larga a coluna com as referências
      eventos = eventos.drop(2, axis='columns')
      # Inverte a ordem as colunas
      eventos = eventos[eventos.columns[::-1]]
      # Atualiza o nome das colunas
      eventos.columns = ['Código', 'Descrição', 'Proventos', 'Descontos']


    for i in range(eventos.shape[0]):
      # Cria uma lista com todos os dados do registro
      registro = [competencia, empregador, codigo_emp, nome_emp, cargo, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Proventos'].iloc[i], eventos['Descontos'].iloc[i]])
      # Adiciona uma nova linha na tabela de registros
      index = len(registros)
      registros.loc[index] = registro

    # Encontra os dados do próximo funcionário
    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if row == 'Data e Assinatura':
        start = i
        break

  # Salva a tabela como xlsx
  nome_arq = f'{nome_arq}.xlsx'
  registros.to_excel(nome_arq, index=False)

  #return registros


# Leitura do arquivo html

In [103]:
# Lê todos os arquivos html e salva seus registros em .xlsx
files = os.listdir()
files = [file for file in files if '.html' in file]
for arq in tqdm(files):
  # Remove o '.html' do nome
  nome = arq[:arq.find('.html')]
  get_registros_from_html(arq, nome)

100%|██████████| 67/67 [03:09<00:00,  2.83s/it]


In [105]:
# Lê todos os arquivos xlsx e junta seus registros
df = pd.DataFrame()
# Lista os arquivos .xlsx
files = os.listdir()
files = [file for file in files if '.xlsx' in file]

for arq in tqdm(files):
  # Salva os DataFrames temporários
  df_temp = pd.read_excel(arq)
  # Concatena os DataFrames
  df = pd.concat([df, df_temp])

df.to_excel('VIGON.xlsx', index=False)

100%|██████████| 63/63 [00:18<00:00,  3.45it/s]


# Limpa o ambiente

In [109]:
def clear(html=True, xlsx=True):
  files = os.listdir()

  if html:
    for file in files:
      if '.html' in file:
        os.remove(file)

  if xlsx:
    for file in files:
      if '.xlsx' in file:
        os.remove(file)